In [1]:
# Load environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.llms import Ollama
from langchain_openai import OpenAI

from langchain.tools import tool
from langchain import hub
from langchain.prompts import load_prompt

from langchain.agents import AgentExecutor, create_react_agent

from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs


In [3]:
# Switch to use openAI
llm = OpenAI(temperature=0, verbose=True, model="gpt-3.5-turbo-instruct") # defautl model : "gpt-3.5-turbo-instruct"
#llm = Ollama(model="mistral")

In [4]:
# custom simple tool
@tool
def square(x: int) -> int:
    """
    squares a number
    """
    return str(int(x)**2)

In [5]:
# tool features
print(square.name)
print(square.description)
print(square.args)

square
square(x: int) -> int - squares a number
{'x': {'title': 'X', 'type': 'integer'}}


In [6]:
# pull prompt from the hub and saves it 
obj = hub.pull("hwchase17/react")
obj.save("prompt.json")

In [7]:
# inititalize prompt and tools for custom agent
prompt = load_prompt('prompt.json')
tools = [
    square
    ]

In [8]:
# the prompt template
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [9]:
agent_custom = create_react_agent(
     llm,
     tools,
    prompt
    )

agent_executor = AgentExecutor(agent=agent_custom, tools=tools, verbose=True)

In [10]:
agent_executor.invoke({"input":"What is the square of 5 "})



> Entering new AgentExecutor chain...
 I should square the number 5
Action: square
Action Input: 52525 is the square of 5
Final Answer: 25

> Finished chain.


{'input': 'What is the square of 5 ', 'output': '25'}

In [11]:
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
chain.invoke(
    "What is the weather like right now in Munich, Germany in degrees Fahrenheit?"
)



> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.5820&hourly=temperature_2m&current_weather=true&temperature_unit=fahrenheit&timezone=auto
{"latitude":48.14,"longitude":11.58,"generationtime_ms":0.06604194641113281,"utc_offset_seconds":3600,"timezone":"Europe/Berlin","timezone_abbreviation":"CET","elevation":521.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°F","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2024-02-28T16:45","interval":900,"temperature":43.5,"windspeed":4.3,"winddirection":24,"is_day":1,"weathercode":3},"hourly_units":{"time":"iso8601","temperature_2m":"°F"},"hourly":{"time":["2024-02-28T00:00","2024-02-28T01:00","2024-02-28T02:00","2024-02-28T03:00","2024-02-28T04:00","2024-02-28T05:00","2024-02-28T06:00","2024-02-28T07:00","2024-02-28T08:00","2024-02-28T09:00","2024-02-28T10:00","2024-02-28T11:00","2024-02-28T12:00","2024-

{'question': 'What is the weather like right now in Munich, Germany in degrees Fahrenheit?',
 'output': ' The current temperature in Munich, Germany is 43.5 degrees Fahrenheit. This information was obtained from the API url: https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.5820&hourly=temperature_2m&current_weather=true&temperature_unit=fahrenheit&timezone=auto.'}